# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  
Входные данные от банка — статистика о платёжеспособности клиентов.Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  
**Цели исследования** - проверка наличия свзей между возвратом кредита в срок и наличием детей, семейным положением, уровнем дохода и целью получения кредита.

## Общая информация о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.duplicated().sum()

54

В датасете представлено 12 столбцов: кол-во детей, трудовой стаж, возраст клиентов, образование и id образования, семейное положение и id семейного положения, пол, тип занятости, задолженность, доход, цель кредита.  
В данных представлена информация о 21 525 клиентах.  
Столбцы представлены следующими типами данных: float, int и object.  
Необходимо удалить пропуски, обработать дубликаты и аномалии, заменить тип данных в столбцах (total_income)

## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заменим пропуски в столбце total_income медианноым значением в зависимости от типа занятости

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Заполним пропуски в столбце **days_employed** медианным значением по каждому типу занятости

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [8]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Пропуски удалены**

## Аномалии

В столбце **days_employed** встречаются отрицательные значения, заменим их на значения по модулю.

In [9]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа days_employed

In [10]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Посмотрим на аномалии в столбце **children**

In [11]:
data.children.unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Удалим строки, где встречаются значения -1 и 20

In [12]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [13]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Посмотрим на распределение по возрасту

In [14]:
data.dob_years.unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

Возраст - категория без сильного разброса, поэтому заменим нулевой возраст на медиану.

In [15]:
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].median())

In [16]:
data[data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


### Изменение типа данных

Заменим тип данных в столбце **total_income** и **days_employed** на целочисленный.

In [17]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].astype(int)

### Обработка дубликатов

In [18]:
data.education.unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведём данные в столбце **education** к нижнему регистру.

In [19]:
data['education'] = data['education'].str.lower()

In [20]:
data.education.unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Удалим остальные дубликаты

In [21]:
data.drop_duplicates(inplace=True)

### Категоризация данных

Создадим столбец **total_income_category** с категориями доходов:  
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [22]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

data['total_income_category'] = data['total_income'].apply(categorize_income)

Посмотрим на цели кредита в столбце **purpose**

In [23]:
data.purpose.unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Все цели делятся на 4 основные:
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Создадим категоризацию

In [24]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование

### Зависимость между кол-вом детей и возвратом кредита в срок

In [25]:
children_debt_dep = data.groupby('children')['debt'].agg(['count', 'sum']) 

children_debt_dep = children_debt_dep.rename(columns={'sum':'debtors', 'count':'loaners'}) 

children_debt_dep['debtors_percent'] = children_debt_dep['debtors']/children_debt_dep['loaners'] 
children_debt_dep['debtors_percent'] = children_debt_dep['debtors_percent'].map('{:.2%}'.format)


children_debt_dep.sort_values(by='debtors_percent', ascending=False) 

,loaners,debtors,debtors_percent
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14090,1063,7.54%
5,9,0,0.00%


Такая группировка не даёт чёткой картины, однако можно отметить, что бездетные реже подвергнуты риску стать должниками.

Посмотрим, сколько всего должников

In [26]:
data['debt'].value_counts()

debt
0    19598
1     1732
Name: count, dtype: int64

Посмотрим на укрупнённую группировку по количеству детей: семьи с 1 ребёнком, семь с 2 детьми, многодетные и бездетные

In [27]:

def categorize_children(children):
    if children>=3:
        return 'многодетные'
    elif children == 0:
        return 'бездетные'
    elif children == 1:
        return 'семьи с 1 ребёнком'
    else:
        return 'семьи с 2 детьми'
    
data['children_categories'] = data['children'].apply(categorize_children) 


children_categories_data = data.groupby('children_categories')['debt'].agg(['sum', 'count'])
children_categories_data = children_categories_data.rename(columns={'sum':'debtors', 'count':'loaners'}) 

children_categories_data['debtors_percent'] = children_categories_data['debtors']/children_categories_data['loaners'] 
children_categories_data['debtors_percent'] = children_categories_data['debtors_percent'].map('{:.2%}'.format)


children_categories_data.sort_values(by='debtors_percent', ascending=False)

,debtors,loaners,debtors_percent
children_categories,,,
семьи с 2 детьми,194,2052,9.45%
семьи с 1 ребёнком,444,4808,9.23%
многодетные,31,380,8.16%
бездетные,1063,14090,7.54%


**Вывод:**  
Наличие детей повышает вероятность задолженности. У семей с 1-2 детьми процент должников выше, чем у многодетных. При этом, очевидно, что у заёмщиков с 2 детьми доля должников выше, чем у заёмщиков с 1 ребёнком.
Это может быть связано с:
1) большей финансовой стабильностью у людей, планирующих содержать 3 и более детей;
2) мерами поддержки многодетных семей;
3) недостаточной статистической значимостью выборки.

### Зависимость между семейным положением и возвратом кредита в срок

In [28]:
famstatus_debt_dep = data.groupby('family_status')['debt'].agg(['count', 'sum'])
famstatus_debt_dep = famstatus_debt_dep.rename(columns={'sum':'debtors', 'count':'loaners'})
famstatus_debt_dep['debtors_percent'] = famstatus_debt_dep['debtors']/famstatus_debt_dep['loaners']
famstatus_debt_dep['debtors_percent'] = famstatus_debt_dep['debtors_percent'].map('{:.2%}'.format)

famstatus_debt_dep.sort_values(by='debtors_percent', ascending=False)

,loaners,debtors,debtors_percent
family_status,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4133,385,9.32%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


Создадим таблицу с категориями: в браке, одинокие

In [29]:

new_data = [
    [16395, 1312], # гражданский брак+женат/замужем
    [4936, 420]    # не женат/не замужем+в разводе+вдовец/вдова
] 

famstatus_data = pd.DataFrame(data=new_data, columns=['loaners', 'debtors'], index=['в браке', 'одиноки'])
famstatus_data['debtors_percent'] = famstatus_data['debtors']/famstatus_data['loaners']
famstatus_data['debtors_percent'] = famstatus_data['debtors_percent'].map('{:.2%}'.format)

famstatus_data

,loaners,debtors,debtors_percent
в браке,16395,1312,8.00%
одиноки,4936,420,8.51%


**Вывод:**  
Категориями с наименьшей долей закредитованности являются вдовцы/вдовы и люди в разводе. Наибольший риск закредитованности выражен у неженатых/незамужних, а также людей в гражданском браке. Возможно здесь присутствует корреляция с возрастом, поскольку неженатые/незамужние и люди в гражданском браке чаще являются молодыми людьми, которые только начинают формировать свой капитал.  
Вторая таблица показывает, что холостые чаще имеют задолженность, чем люди в браке.

### Зависимость между уровнем дохода и возвратом кредита в срок

In [30]:
income_debt_dep = (
                   data
                   .pivot_table(index='total_income_category', values='debt', aggfunc={'sum', 'count'})
                  )

income_debt_dep = income_debt_dep.rename(columns={'sum':'debtors', 'count':'loaners'})
income_debt_dep['debtors_percent'] = income_debt_dep['debtors']/income_debt_dep['loaners']
income_debt_dep['debtors_percent'] = income_debt_dep['debtors_percent'].map('{:.2%}'.format)

income_debt_dep['category_income'] = ['1.000.001 и выше', '200.001–1.000.000', '50.001–200.000', '30.001–50.000', '0–30.000']

income_debt_dep.sort_values(by='debtors_percent', ascending = False) 

,loaners,debtors,debtors_percent,category_income
total_income_category,,,,
E,22,2,9.09%,0–30.000
C,15920,1353,8.50%,50.001–200.000
A,25,2,8.00%,1.000.001 и выше
B,5014,354,7.06%,200.001–1.000.000
D,349,21,6.02%,30.001–50.000


**Вывод:** 
- Наибольшая доля закредитованности принадлежит группе с категорией дохода E (наименее доходная группа). Это может быть связано с тем, что доходов не хватает даже на покрытие необходимых нужд
*Стоит отметить, что количество наблюдений в этой группе крайне мало.
- При этом наименее закредитованной является (следующая за группой E) группа с категорией D (от 30 тыс. до 50 тыс. руб.). Это может быть связано с тем, фактом, что дохода хватает исключительно на покрытие необходимых нужд, при том, что на обслуживание кредита не хватит.
- У категории с наибольшим диапозоном доходов средний процент задолженности. Здесь, вероятно, стоит говорить о незначимости выборки. Количество наблюдений слишком незначительно.
- Исходя из вышесказанного, категория C (условный средний класс) наиболее подвержена возникновению задолженности.
- В целом, доля должников снижается с увеличением дохода

### Влияние цели кредита на возврат в срок

In [31]:
purpose_debt_dep = data.groupby('purpose_category')['debt'].agg(['sum', 'count'])
purpose_debt_dep = purpose_debt_dep.rename(columns={'sum':'debtors', 'count':'loaners'}) # переименуем стобцы для удобства
purpose_debt_dep['debtors_percent'] = purpose_debt_dep['debtors']/purpose_debt_dep['loaners'] # найдём долю должников
purpose_debt_dep['debtors_percent'] = purpose_debt_dep['debtors_percent'].map('{:.2%}'.format)

purpose_debt_dep.sort_values(by='debtors_percent', ascending=False)

,debtors,loaners,debtors_percent
purpose_category,,,
операции с автомобилем,400,4279,9.35%
получение образования,369,3988,9.25%
проведение свадьбы,183,2312,7.92%
операции с недвижимостью,780,10751,7.26%


**Вывод:**
- Заёмщики, берущие кредиты для операций с автомобилями и получения образования, больше остальных подвержены риску появления задолженностей.
- Наименьшая доля должников наблюдается среди заёмщиков, берущих кредиты на операции с недвижимостью.

## Общий вывод

  - Исходные данные содержали пропуски, поэтому на этапе предобработки в столбцах total_income и days_employed осуществлено заполнение пропущенных значений с помощью медианы по категориям для более достоверного исследования (по типу занятости(сотрудник, пенсионер и т.д.). 
  - При этом к значениям столбца days_employed предварительно применено модульное значение для избавления от отрицательных значений. Стоит отметить, что в данном столбце присутствуют аномальные значения (у пенсионеров и безработных), которые трудно исправить. Также аномальные значения были убраны из столбца children (-1, 20). 
  - Для эффективного анализа на этапе предобработки к единому виду были приведены значения из столбца education, а также добавлены столбцы с группировкой заёмщиков по доходам и по цели кредита.
    
Выводы:
1) Наличие детей повышает риск возникновения задолженности.
2) Неженатые/незамужние и люди в гражданском браке более подвержены риску стать должниками. Наименьший риск в вдовцов/вдов и людей в разводе.
3) Группа C (условный средний класс) наиболее подвержен риску стать должниками.
4) Заёмщики, берущие кредиты для операция с автомобилями и получения образования, чаще становятся должниками. Реже всего - те, кто берут кредиты для операций с недвижимостью.
5) Пропуски в исходных данных могут появляться из-за человеческого (ошибки при заполнении, сокрытие информации) и технического(повреждение файла, искажение данных программами) факторов.
6) Наиболее эффективным инструментом для заполнения пропусков является медиана. Она сохраняет центральное значение и менее подвержена выбросами и аномальным значениям. 

Таким образом, наиболее подверженными задолженности группами являются заёмщики с 1-2 детьми, холостые и люди в гражданском браке. Так же стоит обратить внимание на повышенный риск задолженности у заёмщиков, берущих кредиты на операции с автомобилями и получение образования.
    
Для удобства в дальнейших исследованиях стоит внедрить методы для унифицирования заполненных данных. К примеру, вместо свободного заполнения графы education внедрить поле с выбором определённого вида образования в едином стиле.